In [2]:
import numpy as np
import joblib
import os

# Check if models exist before trying to load them
models_exist = all(os.path.exists(f"SEM-{i}_model.pkl") and os.path.exists(f"SEM-{i}_scaler.pkl") for i in range(2, 9))

if not models_exist:
    print("Error: Models not found. Please train the models first.")
    exit()

# Load models and scalers
mlp_models = {}
scalers = {}

print("Loading trained models and scalers...")
for i in range(2, 9):
    target = f"SEM-{i}"
    mlp_models[target] = joblib.load(f"{target}_model.pkl")
    scalers[target] = joblib.load(f"{target}_scaler.pkl")
print("Models loaded successfully!")

# Get input semester
while True:
    try:
        current_sem = int(input("Enter the last semester you have completed (1-7): "))
        if current_sem in range(1, 8):
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 7.")
    except ValueError:
        print("Invalid input. Please enter a numeric value.")

# Collect SGPA for completed semesters
print("\nEnter your SGPA for previous semesters one by one:")
prev_sgpa = []
for i in range(1, current_sem + 1):
    while True:
        try:
            sgpa = float(input(f"Enter SGPA for SEM-{i}: "))
            if 0 <= sgpa <= 10:  # Assuming SGPA is on a scale of 0-10
                prev_sgpa.append(sgpa)
                break
            else:
                print("SGPA should be between 0 and 10. Please try again.")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

# Ask user for prediction preference
while True:
    prediction_choice = input("\nDo you want to predict only the next semester or all remaining semesters? (Enter 'next' or 'all'): ").lower().strip()
    if prediction_choice in ['next', 'all']:
        break
    else:
        print("Invalid choice. Please enter 'next' or 'all'.")

# Determine which semesters to predict
future_semesters = [current_sem + 1] if prediction_choice == 'next' else range(current_sem + 1, 9)

# Make predictions for selected future semesters
print("\nPredictions for Future Semesters:")

# Make a copy of the previous SGPAs to update as we make predictions
all_sgpa = prev_sgpa.copy()

for future_sem in future_semesters:
    target = f"SEM-{future_sem}"

    # Get only the required previous semesters for this prediction
    required_prev_sgpa = all_sgpa[:future_sem-1]

    # Compute derived features
    avg_sgpa = np.mean(required_prev_sgpa)
    sgpa_improvement = np.mean(np.diff(required_prev_sgpa)) if len(required_prev_sgpa) > 1 else 0
    sgpa_difference = required_prev_sgpa[-1] - required_prev_sgpa[0] if len(required_prev_sgpa) > 1 else 0

    # Create feature list matching the training data structure
    features = required_prev_sgpa + [avg_sgpa, sgpa_improvement, sgpa_difference]
    
    # Create input features array
    new_input = np.array(features).reshape(1, -1)

    # Standardize and Predict
    new_input_scaled = scalers[target].transform(new_input)
    prediction = mlp_models[target].predict(new_input_scaled)
    predicted_sgpa = round(prediction[0], 2)

    print(f"Predicted SGPA for {target}: {predicted_sgpa}")

    # Add the prediction to all_sgpa for use in subsequent predictions
    all_sgpa.append(predicted_sgpa)

# Calculate overall predicted CGPA if all semesters were predicted
if prediction_choice == 'all' and current_sem < 8:
    predicted_cgpa = round(sum(all_sgpa) / 8, 2)
    print(f"\nPredicted final CGPA after 8 semesters: {predicted_cgpa}")

Loading trained models and scalers...
Models loaded successfully!


Enter the last semester you have completed (1-7):  4



Enter your SGPA for previous semesters one by one:


Enter SGPA for SEM-1:  8.06
Enter SGPA for SEM-2:  7.79
Enter SGPA for SEM-3:  8.32
Enter SGPA for SEM-4:  7.89

Do you want to predict only the next semester or all remaining semesters? (Enter 'next' or 'all'):  all



Predictions for Future Semesters:
Predicted SGPA for SEM-5: 8.17
Predicted SGPA for SEM-6: 7.98
Predicted SGPA for SEM-7: 7.94
Predicted SGPA for SEM-8: 7.78

Predicted final CGPA after 8 semesters: 7.99


C:\Users\Noyal\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Noyal\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Noyal\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Noyal\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
